### Load modules

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import KFold

In [3]:
import h5py

### Load train image list

In [19]:
df = pd.read_csv('driver_imgs_list.csv')
# drivers' names
subjects = np.unique(df['subject'])
# classes
classnames = np.unique(df['classname'])

### 5-fold train-val split in terms of drivers: 

In [20]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
subjects_t = []
subjects_v = []
for sub_t, sub_v in kf.split(subjects):
    subjects_t.append(list(subjects[sub_t]))
    subjects_v.append(list(subjects[sub_v]))

split the train image list

In [21]:
def split_list(subjects_t, subjects_v):
    df_t = df.loc[df.subject.isin(subjects_t)]
    df_v = df.loc[df.subject.isin(subjects_v)]
    return df_t, df_v

### original dataset dir.

In [9]:
dataset_dir = '/dataset_dir'
# dataset_dir = '../../dataset_resized'

train_raw_dir = dataset_dir + '/train'
test_dir = dataset_dir + '/test'

### make dirs of train and validation

In [8]:
import shutil
from shutil import *

In [9]:
from tqdm import tqdm

In [10]:
def mkdir_train_val(fold):
    if 'train'+str(fold) in os.listdir():
        shutil.rmtree('train'+str(fold))
    if 'validation'+str(fold) in os.listdir():
        shutil.rmtree('validation'+str(fold))
    for split_name in ['train', 'validation']:
        for cls in classnames:
            os.makedirs(split_name+str(fold) + '/' + cls)

In [11]:
def symlink_train_val(df_t, df_v, fold, train_dir):
    mkdir_train_val(fold)
    for i in tqdm(range(len(df_t))):
        cls = df_t.iloc[i]['classname']
        img = df_t.iloc[i]['img']
        os.symlink(train_dir+'/'+cls+'/'+img, 'train'+str(fold)+'/'+cls+'/'+img)
    for i in tqdm(range(len(df_v))):
        cls = df_v.iloc[i]['classname']
        img = df_v.iloc[i]['img']
        os.symlink(train_dir+'/'+cls+'/'+img, 'validation'+str(fold)+'/'+cls+'/'+img)

In [12]:
def copy_train_val(df_t, df_v, fold, train_dir):
    mkdir_train_val(fold)
    for i in tqdm(range(len(df_t))):
        cls = df_t.iloc[i]['classname']
        img = df_t.iloc[i]['img']
        copyfile(train_dir+'/'+cls+'/'+img, 'train'+str(fold)+'/'+cls+'/'+img)
    for i in tqdm(range(len(df_v))):
        cls = df_v.iloc[i]['classname']
        img = df_v.iloc[i]['img']
        copyfile(train_dir+'/'+cls+'/'+img, 'validation'+str(fold)+'/'+cls+'/'+img)

In [13]:
for fold in range(4,5):
    df_t, df_v = split_list(subjects_t[fold], subjects_v[fold])
    symlink_train_val(df_t, df_v, fold, train_raw_dir)

100%|██████████| 4739/4739 [00:39<00:00, 120.25it/s]


In [14]:
train_dir = 'train4'
val_dir = 'validation4'

### preprocessing images

In [4]:
# preprocess function of VGG and ResNet50
def preprocess_fn(x):
    # RGB >> BGR
    x = img_to_array(x)
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [5]:
# default image size of VGG and ResNet50
img_width, img_height = 224, 224

In [6]:
batch_size = 32

In [7]:
data_gen = ImageDataGenerator(preprocessing_function=preprocess_fn)

original generator

In [22]:
train_generator = data_gen.flow_from_directory(train_dir, 
                                               target_size=(img_height, img_width), 
                                               class_mode='categorical',
                                               batch_size=batch_size)

Found 17446 images belonging to 10 classes.


In [19]:
val_generator = data_gen.flow_from_directory(val_dir, 
                                             target_size=(img_height, img_width), 
                                             class_mode='categorical',
                                             batch_size=batch_size)

Found 4739 images belonging to 10 classes.


In [10]:
test_generator = data_gen.flow_from_directory(test_dir, 
                                              target_size=(img_height, img_width), 
                                              class_mode=None,
                                              shuffle=False,
                                              batch_size=batch_size)

Found 79726 images belonging to 1 classes.


transfer generator

In [ ]:
train_tf_generator = data_gen.flow_from_directory(train_dir, 
                                                  target_size=(img_height, img_width), 
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=batch_size)

In [ ]:
val_tf_generator = data_gen.flow_from_directory(val_dir,
                                                target_size=(img_height, img_width), 
                                                class_mode='categorical',
                                                shuffle=False,
                                                batch_size=batch_size)

augment generator

In [20]:
data_gen_aug = ImageDataGenerator(rotation_range=30.,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  preprocessing_function=preprocess_fn)

In [21]:
train_aug_generator = data_gen_aug.flow_from_directory(train_dir, 
                                                       target_size=(img_height, img_width), 
                                                       class_mode='categorical',
                                                       batch_size=batch_size)

Found 17685 images belonging to 10 classes.


In [42]:
val_aug_generator = data_gen_aug.flow_from_directory(val_dir, 
                                                     target_size=(img_height, img_width), 
                                                     class_mode='categorical',
                                                     batch_size=batch_size)

Found 4978 images belonging to 10 classes.


In [44]:
test_aug_generator = data_gen_aug.flow_from_directory(test_dir, 
                                                      target_size=(img_height, img_width), 
                                                      class_mode=None,
                                                      shuffle=False,
                                                      batch_size=batch_size)

Found 79726 images belonging to 1 classes.


### Fine-tuning VGG16

In [11]:
from keras.optimizers import *
from keras.callbacks import *

In [12]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

In [13]:
def model_build(MODEL, optimizer, layer_num_fix, top_dropout=False):
    base_model = MODEL(weights='imagenet', include_top=True)
    if top_dropout:
        x = Dropout(0.4, name='top_dropout')(base_model.layers[-2].output)
        x = Dense(10, activation='softmax', name='top_prediction')(x)
    else:
        x = Dense(10, activation='softmax', name='top_prediction')(base_model.layers[-2].output)
        
    model_ft = Model(base_model.input, x, name=MODEL.__name__)
    model_ft.compile(optimizer=optimizer, 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    for layer in model_ft.layers[:layer_num_fix]:
        layer.trainable = False
    return model_ft

In [14]:
model_vgg16_ft = model_build(VGG16, adam, 10, False)

In [29]:
tensorboard_cb = TensorBoard(log_dir='log/vgg16', histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)
ckpt_cb = ModelCheckpoint('vgg16_ft4_aug_weights.{epoch:02d}-{val_loss:.3f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
num_epochs = 50

In [ ]:
model_vgg16_ft.fit_generator(train_aug_generator, 
                             50,
                             num_epochs, 
                             validation_data=val_generator, 
                             validation_steps=val_generator.samples//batch_size+1,
                             callbacks=[tensorboard_cb, ckpt_cb])

Epoch 1/50
50/50 [==============================] - 218s - loss: 2.5061 - acc: 0.1044 - val_loss: 2.3101 - val_acc: 0.1042
Epoch 2/50
50/50 [==============================] - 191s - loss: 2.3032 - acc: 0.1225 - val_loss: 2.2698 - val_acc: 0.1618
Epoch 3/50
50/50 [==============================] - 193s - loss: 2.2610 - acc: 0.1450 - val_loss: 2.1093 - val_acc: 0.2271
Epoch 4/50
50/50 [==============================] - 195s - loss: 1.9141 - acc: 0.2575 - val_loss: 1.7272 - val_acc: 0.3440
Epoch 5/50
50/50 [==============================] - 193s - loss: 1.5865 - acc: 0.3956 - val_loss: 1.5405 - val_acc: 0.3760
Epoch 6/50
50/50 [==============================] - 193s - loss: 1.3754 - acc: 0.4619 - val_loss: 1.1858 - val_acc: 0.4752
Epoch 7/50
50/50 [==============================] - 181s - loss: 1.2125 - acc: 0.5112 - val_loss: 1.2809 - val_acc: 0.5664
Epoch 8/50
50/50 [==============================] - 193s - loss: 1.0276 - acc: 0.5938 - val_loss: 0.9602 - val_acc: 0.6204
Epoch 9/50
50/50

50/50 [==============================] - 181s - loss: 0.1034 - acc: 0.9675 - val_loss: 0.3990 - val_acc: 0.8761
Epoch 33/50
50/50 [==============================] - 181s - loss: 0.1084 - acc: 0.9669 - val_loss: 0.6300 - val_acc: 0.8447
Epoch 34/50
50/50 [==============================] - 181s - loss: 0.1199 - acc: 0.9637 - val_loss: 0.3719 - val_acc: 0.8848
Epoch 35/50
50/50 [==============================] - 181s - loss: 0.0919 - acc: 0.9750 - val_loss: 0.4039 - val_acc: 0.8778
Epoch 36/50
50/50 [==============================] - 179s - loss: 0.0815 - acc: 0.9831 - val_loss: 0.5779 - val_acc: 0.8061
Epoch 37/50
50/50 [==============================] - 181s - loss: 0.0716 - acc: 0.9788 - val_loss: 0.3626 - val_acc: 0.8890
Epoch 38/50
49/50 [============================>.] - ETA: 1s - loss: 0.0761 - acc: 0.9790

KeyboardInterrupt: 

In [21]:
model_vgg16_ft.load_weights('vgg16_ft4_aug_weights.15-0.241.hdf5')

In [17]:
def write_predicts_gen(test_gen, batch_size, sub_fname):
    y_pred = model_vgg16_ft.predict_generator(test_gen, 
                                              test_gen.samples//batch_size+1, 
                                              verbose=1)

    y_pred_df = pd.DataFrame(y_pred, columns=classnames)
    
    img_name = []
    for i, fname in enumerate(test_gen.filenames):
        img_name.append(os.path.basename(fname))

    img_name_df = pd.DataFrame(img_name, columns=['img'])
    predictions = pd.concat([img_name_df, y_pred_df], axis=1)
    print(predictions.shape)
    predictions.to_csv(sub_fname, index=False)
    print('submission file {} is ready.'.format(sub_fname))

In [ ]:
write_predicts_gen(test_generator, batch_size, 'submission_vgg16_ft4_aug.csv')

2492/2492 [==============================] - 1473s  
(79726, 11)
submission file submission_vgg16_ft4_aug.csv is ready.


In [15]:
model_vgg16_ft.load_weights('vgg16_ft4_aug_weights.10-0.380.hdf5')

In [ ]:
write_predicts_gen(test_generator, batch_size, 'submission_vgg16_ft4_aug_0.380.csv')

1735/2492 [===================>..........] - ETA: 425s